# Features - Fake news detection

In [1]:
import spacy
import os
import json
import numpy as np
import pandas as pd
import pickle

from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from spacy_readability import Readability
from spacy.language import Language

nlp = spacy.load("fr_dep_news_trf")

def get_readability(nlp, name):
    read = Readability()
    return read

Language.factory("my_readability", func=get_readability)
nlp.add_pipe("my_readability", last=True)

## Dataset

In [2]:
def remove_useless(paragraph_ann):
    content = paragraph_ann["content"]
    paragraphs_to_delete = []
    entities_to_delete = []

    for index, paragraph in enumerate(content) :
        if 'label' in paragraph.keys() and paragraph["label"]==0:
            paragraphs_to_delete.append(index)

        else :
            if type(paragraph["content"])==list:
              for index_entity, entity_content in enumerate(paragraph["content"]):
                if type(entity_content) == dict and 'label' in entity_content.keys() and entity_content["label"]==0:
                      entities_to_delete.append((index, index_entity))

    for i in range(-1, -len(entities_to_delete)-1, -1) :
      index, index_entity = entities_to_delete[i]
      del paragraph_ann['content'][index]['content'][index_entity]

    for index in reversed(paragraphs_to_delete):
      del paragraph_ann['content'][index]

    return paragraph_ann


def fusion(paragraph_ann):
  label = paragraph_ann['label']
  author = paragraph_ann['author']
  title = paragraph_ann['title']
  date = paragraph_ann['date']
  content = ""
  for paragraph in paragraph_ann['content']:
    if type(paragraph['content']) == str :
      content += paragraph['content']
    elif type(paragraph['content']) == list :
      for entity in paragraph['content']:
        content += entity['content']
  json = {
    'label' : label, 
    'date': date,
    'title': title,
    'author': author,
    'content' : content,
  }
  return json

In [3]:
# assign directory
# ANNOTATIONS_FOLDER = 'C:\\Users\\louis\\Desktop\\NLP\\fake_news\\annotations'
# dataset = []
 
# # iterate over files in
# # that directory
# for filename in os.listdir(ANNOTATIONS_FOLDER):
#     f = os.path.join(ANNOTATIONS_FOLDER, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         with open(f, 'r') as file:
#             data = json.load(file)
#             if data["label"] > 0:
#                 data = fusion(remove_useless(data))
#                 dataset += [(data["label"], data["content"])]

# dataset = np.array(dataset, dtype=str)

# PATH = 'text_dataset.csv'
# fields = ['label', 'article']
# dataset = np.array(pd.read_csv(PATH, usecols=fields), dtype=str)

# print(f'Dataset: {dataset.shape}')

labels = np.array(pickle.load(open("labels_train.p", "rb")), dtype=object)
docs = np.array(pickle.load(open("docs_train.p", "rb")), dtype=object)

train_dataset = np.vstack((labels, docs)).T

np.random.shuffle(train_dataset)

print(f'Dataset: {train_dataset.shape}')

Dataset: (1053, 2)


In [4]:
x
# y_train, x_train_txt = np.array(train[:,0], dtype=int), [str(txt) for txt in train[:,1]]
# y_test, x_test_txt = np.array(test[:, 0], dtype=int), [str(txt) for txt in test[:,1]]
# print(f'Train: {len(x_train_txt)} ({type(x_train_txt[0])})')
# print(f'Test: {len(x_test_txt)}')
y_train, x_train_doc = np.array(train_dataset[:,0], dtype=int), list(train_dataset[:, 1])
y_test, x_test_doc = pickle.load(open("labels_test.p", "rb")), pickle.load(open("docs_test.p", "rb"))

print(f'Train: {len(x_train_doc)} ({type(x_train_doc[0])})')
print(f'Test: {len(x_test_doc)}')

Train: 789 (<class 'str'>)
Test: 264


In [19]:
# x_train_doc = list(nlp.pipe(x_train_txt))
# x_test_doc = list(nlp.pipe(x_test_txt))

## Features

### Functions

In [16]:
def get_punct_ratio(doc) -> float:
    n_token = len(doc)
    if n_token:
        return sum([1 if token.pos_ == "PUNCT" else 0 for token in doc]) / n_token
    else:
        return .0

def get_adv_ratio(doc) -> float:
    n_token = len(doc)
    if n_token:
        return sum([1 if token.pos_ == "ADV" else 0 for token in doc]) / n_token
    else:
        return .0

def get_fin_ratio(doc) -> float:
    n_token = len(doc)
    if n_token:
        return sum([1 if "Fin" in token.morph.get("VerbForm") else 0 for token in doc]) / n_token
    else:
        return .0   

def get_fkre(doc) -> float:
    return doc._.flesch_kincaid_reading_ease

def get_length(doc) -> float:
    return len(doc)

def get_expr(doc) -> float:
    return sum([1 if token.lemma_ in ['?', '!', '(', ')'] else 0 for token in doc])

In [17]:
FEATURES = {
    "PUNCT": get_punct_ratio,
    "ADV": get_adv_ratio,
    "FIN": get_fin_ratio,
    "FKRE": get_fkre,
    "LENGTH": get_length,
    "EXPR": get_expr,
}

features_train = np.zeros((len(x_train_doc), len(FEATURES)), dtype=float)
features_test = np.zeros((len(x_test_doc), len(FEATURES)), dtype=float)

### Computation

In [18]:
print("Generating features for train...")
for doc_idx, doc in enumerate(x_train_doc):
    for feature_idx, feature in enumerate(FEATURES.keys()):
        get_feature = FEATURES[feature]
        features_train[doc_idx, feature_idx] = get_feature(doc)

C:\Users\louis\miniconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3204096 bytes.

In [ ]:
print("Generating features for test...")
for doc_idx, doc in enumerate(x_test_doc):
    for feature_idx, feature in enumerate(FEATURES.keys()):
        get_feature = FEATURES[feature]
        features_test[doc_idx, feature_idx] = get_feature(doc)

In [ ]:
classifier = SVC(class_weight='balanced')
print("Fitting classifier")
classifier.fit(features_train, y_train)
print("Predicting...")
y_predicted = np.array(classifier.predict(features_test), dtype=int)

## Scores

In [ ]:
def evaluate(predictions, labels, metric='report'):
    if metric == 'report':
        return metrics.classification_report(labels, predictions, zero_division=0)
    elif metric == 'matrix':
        return metrics.confusion_matrix(labels, predictions)

In [ ]:
report = evaluate(y_predicted, y_test)
print(report)
matrix = evaluate(y_predicted, y_test, metric="matrix")
print(matrix)